In [1]:
%display latex
from pylie import *
from pprint import pprint 
from collections.abc import Iterable
import functools
from operator import mul, sub
from pprint import pprint
from IPython.core.debugger import set_trace


In [2]:
def Lfunc(e):
    return e

In [3]:
def Lcml(e):
    '''Least common left multiply, p.23'''
    return 0

In [4]:
def Lcm(e):
    return 0

In [5]:
def CompleteSystem(S, context):
    return S

In [6]:
def FindIntegrableConditions(S, context):
    return S

In [7]:
def ReduceIntegrabilityConditions(S, context):
    return S

In [8]:
var ("x y")
z = function("z")(x,y)
w = function("w")(x,y)
ctx = Context((w,z), (x,y), MatrixOrder.Mgrlex)
f1 =diff(w, y) + x*diff(z,y)/(2*y*(x**2+y)) - w(x,y)/y
f1.show()

-w(x, y)/y + 1/2*x*diff(z(x, y), y)/((x^2 + y)*y) + diff(w(x, y), y)

In [9]:
f2=diff(z,x,y) + y*diff(w,y)/x + 2*y*diff(z, x)/x
f2.show()

y*diff(w(x, y), y)/x + 2*y*diff(z(x, y), x)/x + diff(z(x, y), x, y)

In [10]:
f3 = diff(w, x,y) - 2*x*diff(z, x,2)/y - x*diff(w,x)/y**2
f3.show()

-2*x*diff(z(x, y), x, x)/y - x*diff(w(x, y), x)/y^2 + diff(w(x, y), x, y)

In [11]:
f4 = diff(w, x,y) + diff(z, x,y) + diff(w, y)/(2*y) - diff(w,x)/y + x* diff(z, y)/y - w(x,y)/(2*y**2)
f4.show()

x*diff(z(x, y), y)/y - diff(w(x, y), x)/y + 1/2*diff(w(x, y), y)/y - 1/2*w(x, y)/y^2 + diff(w(x, y), x, y) + diff(z(x, y), x, y)

In [12]:
f5 =diff(w,y,y) + diff(z,x,y) - diff(w, y)/y + w(x,y)/(y**2)
f5.show()

-diff(w(x, y), y)/y + w(x, y)/y^2 + diff(w(x, y), y, y) + diff(z(x, y), x, y)

In [13]:
#f2a=f2-(y*f1/x);f2a.expand().show()

In [14]:
f1dp=Differential_Polynomial(f1, context=ctx)
f2dp=Differential_Polynomial(f2, context=ctx)
f3dp=Differential_Polynomial(f3, context=ctx)
f4dp=Differential_Polynomial(f4, context=ctx)
f5dp=Differential_Polynomial(f5, context=ctx)

In [15]:
ro = Reorder ([f1dp, f2dp, f3dp, f4dp, f5dp], ctx)

In [16]:
for _r in ro:
    _r.show()

-2*x*diff(z(x, y), x, x)/y - x*diff(w(x, y), x)/y^2 + diff(w(x, y), x, y)

x*diff(z(x, y), y)/y - diff(w(x, y), x)/y + 1/2*diff(w(x, y), y)/y - 1/2*w(x, y)/y^2 + diff(w(x, y), x, y) + diff(z(x, y), x, y)

-diff(w(x, y), y)/y + w(x, y)/y^2 + diff(w(x, y), y, y) + diff(z(x, y), x, y)

y*diff(w(x, y), y)/x + 2*y*diff(z(x, y), x)/x + diff(z(x, y), x, y)

1/2*x*diff(z(x, y), y)/(x^2*y + y^2) - w(x, y)/y + diff(w(x, y), y)

In [17]:
def JanetBasis(S, context):
    """List of homogenous PDE's + order context"""
    if not isinstance(S, Iterable):
        # bad criterion
        S = [S]
    S = Reorder([Differential_Polynomial(s, context) for s in S], context, ascending = True)
    while 1:
        S = Autoreduce (S, context)        
        #f = FindIntegrableConditions(S, context)
        #c = ReduceIntegrabilityConditions(S, context)
        #if all ([_ == 0 for _ in c]) or not c:
        #    return S
        #S += c
        #S = Reorder (S, context)
        return S[:]

In [18]:
checkS=JanetBasis([f3,f1,f5, f2,f4], ctx)

In [19]:
for _ in checkS:
    _.show()

diff(z(x, y), y)

1/2*w(x, y)/y + diff(z(x, y), x)

-w(x, y)/y + diff(w(x, y), y)

diff(w(x, y), x)

In [23]:
def in_class (r, mclass, M, vars):
    '''checks whether "r" is in the same class like "mclass"'''
    mult, nonmult = vec_multipliers (mclass, M , vars)
    return all (vec_degree (x, r) >= vec_degree (x, mclass) for x in mult)  and \
        all (vec_degree(x, r) == vec_degree (x, mclass) for x in nonmult)

def complete (l,ctx):
    while True:
        m0 = []
        for m in l :
            # XXX FixMe
            _, nonmult = vec_multipliers (m , l , (2,1,0))
            for nm in nonmult:
                _m = list(m)
                _m [nm] += 1
                # XXX Fixme
                if not any (in_class (tuple(_m), v, l, (2,1,0)) for v in l):
                    m0.append (tuple(_m))
        if set(m0) == set() or set(m0) == set(l):
            return l
        l.extend (m0)
        l = reversed(sorted(map (lambda _ : Differential_Vector(_, ctx), list(set(l)))))
        l = [_._e for _ in l] 


def CompleteSystem(S, context):
    s = {}
    for _ in S:
        _fun = _.Lder().operator().function()
        s.setdefault (_fun, []).append (_)
    for k in s:
        _ = complete (s[k], context)
        print (_)
    pprint (s)
    return S

In [24]:
S = CompleteSystem (checkS, ctx)

TypeError: 'Differential_Polynomial' object is not subscriptable